In [14]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

from tensorflow.keras.layers import Dense
from tensorflow.keras import models
from tensorflow.keras import callbacks
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from IPython.display import display

In [60]:
input_dir="../input/efficientnet_output_straight/"
dataset_dir="../input/ranzcr-clip-catheter-line-classification/"

input_df=pd.read_csv("./models/efficientnet_output_normalized.csv")
display(input_df)

,StudyInstanceUID,0,1,2,3,4,5,6,7,8,...,2550,2551,2552,2553,2554,2555,2556,2557,2558,2559
0,1.2.826.0.1.3680043.8.498.26697628953273228189...,0.169359,0.324016,0.173553,0.095382,0.063119,0.165738,0.181906,0.437814,0.098587,...,0.302277,0.070963,0.354162,0.284205,0.057408,0.101549,0.200983,0.098618,0.276763,0.473490
1,1.2.826.0.1.3680043.8.498.46302891597398758759...,0.365546,0.175262,0.286057,0.205560,0.388747,0.452293,0.021495,0.460131,0.311742,...,0.232927,0.223938,0.135535,0.042117,0.169312,0.383767,0.095033,0.151809,0.048868,0.117311
2,1.2.826.0.1.3680043.8.498.23819260719748494858...,0.211337,0.303350,0.184402,0.248883,0.202594,0.301177,0.208046,0.416033,0.378476,...,0.370746,0.315548,0.384991,0.181793,0.134341,0.147335,0.313144,0.356484,0.112562,0.404958
3,1.2.826.0.1.3680043.8.498.68286643202323212801...,0.520994,0.385111,0.367907,0.247489,0.097945,0.448468,0.377076,0.108094,0.598928,...,0.355882,0.156018,0.316916,0.141020,0.041345,0.297808,0.273713,0.329452,0.203884,0.219169
4,1.2.826.0.1.3680043.8.498.10050203009225938259...,0.383775,0.214603,0.263212,0.429333,0.619226,0.365313,0.264323,0.201577,0.515184,...,0.302169,0.318546,0.097627,0.337127,0.124034,0.266738,0.595066,0.150074,0.454537,0.071115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30078,1.2.826.0.1.3680043.8.498.74257566841157531124...,0.152407,0.273989,0.429204,0.095627,0.143194,0.151159,0.190642,0.151711,0.444210,...,0.417215,0.330231,0.257251,0.193722,0.070763,0.415085,0.388353,0.156808,0.239679,0.161568
30079,1.2.826.0.1.3680043.8.498.46510939987173529969...,0.303968,0.299640,0.232293,0.115276,0.309395,0.338309,0.164800,0.221858,0.366292,...,0.509008,0.143263,0.214838,0.167794,0.075009,0.163606,0.415589,0.059742,0.148673,0.164074
30080,1.2.826.0.1.3680043.8.498.43173270582850645437...,0.316939,0.351076,0.589629,0.291154,0.185748,0.232997,0.113028,0.353113,0.366704,...,0.542412,0.236196,0.170918,0.283692,0.270579,0.275420,0.285688,0.203561,0.118533,0.222272
30081,1.2.826.0.1.3680043.8.498.95092491950130838685...,0.239194,0.393650,0.438603,0.363903,0.144577,0.259132,0.366278,0.193645,0.443432,...,0.387640,0.075916,0.316283,0.119015,0.101108,0.645644,0.305596,0.319096,0.330187,0.278469


In [58]:
def normalize(input_df):
    scaler=MinMaxScaler()
    X=scaler.fit_transform(input_df.iloc[:,1:])
    X_df=pd.DataFrame(X.astype(np.float32))
    input_df=pd.concat([input_df["StudyInstanceUID"],X_df],axis=1)
    input_df.to_csv("./models/efficientnet_output_normalized.csv",index=False)

In [61]:
input_shape=(2560,)
encoding_dim=100

model=models.Sequential([
    Dense(encoding_dim*4,activation="relu",input_shape=input_shape),
    Dense(encoding_dim,activation="relu",),
    Dense(encoding_dim*4,activation="relu"),    
    Dense(input_shape[0],activation="sigmoid")
])

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 400)               1024400   
_________________________________________________________________
dense_20 (Dense)             (None, 100)               40100     
_________________________________________________________________
dense_21 (Dense)             (None, 400)               40400     
_________________________________________________________________
dense_22 (Dense)             (None, 2560)              1026560   
Total params: 2,131,460
Trainable params: 2,131,460
Non-trainable params: 0
_________________________________________________________________


In [62]:
X_train, X_test=train_test_split(input_df.iloc[:,1:])

adam=Adam(learning_rate=0.0001)
model.compile(optimizer=adam,loss="mse")

fit_callbacks=[callbacks.EarlyStopping(monitor="val_loss",patience=20,mode="min")]

model.fit(X_train,X_train,epochs=200,shuffle=True,validation_data=(X_test,X_test),callbacks=fit_callbacks)

score=model.evaluate(X_test,X_test,verbose=0)
print(f"\nscore: {score}")

Epoch 1/200
 53/706 [=>............................] - ETA: 22s - loss: 0.0498

KeyboardInterrupt: 